<a href="https://colab.research.google.com/github/Rathorerishabh/PRODIGY_DS_01/blob/main/Resaturent_Text_Review_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from imblearn.over_sampling import SMOTE
import regex as re

In [13]:
df=pd.read_csv("train.csv")

In [14]:
df

,ID,RecipeNumber,RecipeCode,RecipeName,CommentID,UserID,UserName,UserReputation,CreationTimestamp,ReplyCount,ThumbsUpCount,ThumbsDownCount,Rating,BestScore,Recipe_Review
0,70,71,12259,Banana Bars with Cream Cheese Frosting,sp_aUSaElGf_12259_c_149978,u_1oKVaBmiEKwAFyiHrDLoWpbel0J,misscoffeepot,20,1622717078,0,0,0,5,100,great recipe! I have made them just as written...
1,72,88,8202,Simple Taco Soup,sp_aUSaElGf_8202_c_310332,u_1oKZeRIXYzGNhGW8OMR3O4lX1ih,MichelleMalone,1,1622717852,0,0,1,5,100,This is an easy and quick recipe that is great...
2,458,3,2832,Cheeseburger Soup,sp_aUSaElGf_2832_c_206522,u_1oKYHUtdaD48KJCDYq5wfpAUxWz,jeannem32,1,1622717357,0,0,0,3,100,I think I was looking for something extra spec...
3,7,50,100276,Grilled Huli Huli Chicken,sp_aUSaElGf_100276_c_434088,u_1oKZCQcKQZdQhDVBS7oISc216VA,ds422,1,1622648899,0,0,0,5,100,This is our favorite grilled chicken recipe. ...
4,60,12,19731,Cauliflower Soup,sp_aUSaElGf_19731_c_387011,u_1oKd4sudZQPgu6YALiC7bW7mKMs,SusanHoward,1,1622718260,0,0,0,4,100,Great basic recipe. I added 2 teaspoons Tuscan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13631,115,98,11767,Quick Cream of Mushroom Soup,sp_aUSaElGf_11767_c_127710,u_1oKVbHjirkAIWwTjbt9dWyqENo2,elbay,1,1622716969,0,0,0,5,100,Delicious!! Very easy to make! One minor chang...
13632,372,6,21444,Favorite Chicken Potpie,sp_aUSaElGf_21444_c_196030,u_1oKXoBnyJ2ocquNYwAhs19LMSrA,PauletteInhofe,1,1622717329,0,0,0,5,100,I made this pie for my family and they loved i...
13633,87,37,20170,Taco Lasagna,sp_aUSaElGf_20170_c_359338,u_1oKbGr5o9tj0mQl1rkVqzWnE481,Barbaralarkin,1,1622718089,0,0,0,5,100,Very tasty!
13634,48,63,24886,Chicken and Dumplings,sp_aUSaElGf_24886_c_175514,u_1oKWD7Dqjmc0oYHdW8Vm3fzhxJW,SammyPrissy,1,1622717192,0,0,0,5,100,I have made this numerous times. It&#39;s so ...


In [15]:
df.isnull().sum()

ID                   0
RecipeNumber         0
RecipeCode           0
RecipeName           0
CommentID            0
UserID               0
UserName             0
UserReputation       0
CreationTimestamp    0
ReplyCount           0
ThumbsUpCount        0
ThumbsDownCount      0
Rating               0
BestScore            0
Recipe_Review        2
dtype: int64

In [18]:
df=df.dropna()
df

ValueError: For argument "inplace" expected type bool, received type str.

In [17]:
df.isnull().sum()

ID                   0
RecipeNumber         0
RecipeCode           0
RecipeName           0
CommentID            0
UserID               0
UserName             0
UserReputation       0
CreationTimestamp    0
ReplyCount           0
ThumbsUpCount        0
ThumbsDownCount      0
Rating               0
BestScore            0
Recipe_Review        0
dtype: int64

In [19]:
df

,ID,RecipeNumber,RecipeCode,RecipeName,CommentID,UserID,UserName,UserReputation,CreationTimestamp,ReplyCount,ThumbsUpCount,ThumbsDownCount,Rating,BestScore,Recipe_Review
0,70,71,12259,Banana Bars with Cream Cheese Frosting,sp_aUSaElGf_12259_c_149978,u_1oKVaBmiEKwAFyiHrDLoWpbel0J,misscoffeepot,20,1622717078,0,0,0,5,100,great recipe! I have made them just as written...
1,72,88,8202,Simple Taco Soup,sp_aUSaElGf_8202_c_310332,u_1oKZeRIXYzGNhGW8OMR3O4lX1ih,MichelleMalone,1,1622717852,0,0,1,5,100,This is an easy and quick recipe that is great...
2,458,3,2832,Cheeseburger Soup,sp_aUSaElGf_2832_c_206522,u_1oKYHUtdaD48KJCDYq5wfpAUxWz,jeannem32,1,1622717357,0,0,0,3,100,I think I was looking for something extra spec...
3,7,50,100276,Grilled Huli Huli Chicken,sp_aUSaElGf_100276_c_434088,u_1oKZCQcKQZdQhDVBS7oISc216VA,ds422,1,1622648899,0,0,0,5,100,This is our favorite grilled chicken recipe. ...
4,60,12,19731,Cauliflower Soup,sp_aUSaElGf_19731_c_387011,u_1oKd4sudZQPgu6YALiC7bW7mKMs,SusanHoward,1,1622718260,0,0,0,4,100,Great basic recipe. I added 2 teaspoons Tuscan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13631,115,98,11767,Quick Cream of Mushroom Soup,sp_aUSaElGf_11767_c_127710,u_1oKVbHjirkAIWwTjbt9dWyqENo2,elbay,1,1622716969,0,0,0,5,100,Delicious!! Very easy to make! One minor chang...
13632,372,6,21444,Favorite Chicken Potpie,sp_aUSaElGf_21444_c_196030,u_1oKXoBnyJ2ocquNYwAhs19LMSrA,PauletteInhofe,1,1622717329,0,0,0,5,100,I made this pie for my family and they loved i...
13633,87,37,20170,Taco Lasagna,sp_aUSaElGf_20170_c_359338,u_1oKbGr5o9tj0mQl1rkVqzWnE481,Barbaralarkin,1,1622718089,0,0,0,5,100,Very tasty!
13634,48,63,24886,Chicken and Dumplings,sp_aUSaElGf_24886_c_175514,u_1oKWD7Dqjmc0oYHdW8Vm3fzhxJW,SammyPrissy,1,1622717192,0,0,0,5,100,I have made this numerous times. It&#39;s so ...


In [28]:
def preprocess_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    # Remove punctuation and special characters
    text = re.sub(r'[!#&;?-]', '', text)
    return text

In [29]:
df['Recipe_Review'] = df['Recipe_Review'].apply(preprocess_text)

<ipython-input-29-469a9d78a474>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Recipe_Review'] = df['Recipe_Review'].apply(preprocess_text)


In [25]:
df["Recipe_Review"]

0        great recipe I have made them just as written,...
1        This is an easy and quick recipe that is great...
2        I think I was looking for something extra spec...
3        This is our favorite grilled chicken recipe.  ...
4        Great basic recipe. I added 2 teaspoons Tuscan...
                               ...                        
13631    Delicious Very easy to make One minor change ....
13632    I made this pie for my family and they loved i...
13633                                           Very tasty
13634    I have made this numerous times.  It39s so goo...
13635    Super. I added 1/2 lb chopped bacon to mine. H...
Name: Recipe_Review, Length: 13634, dtype: object

In [30]:
df.dtypes

ID                    int64
RecipeNumber          int64
RecipeCode            int64
RecipeName           object
CommentID            object
UserID               object
UserName             object
UserReputation        int64
CreationTimestamp     int64
ReplyCount            int64
ThumbsUpCount         int64
ThumbsDownCount       int64
Rating                int64
BestScore             int64
Recipe_Review        object
dtype: object

In [31]:
label_encoders = {}
for column in df.select_dtypes(include=['object']).columns:
    label_encoders[column] = LabelEncoder()
    df[column] = label_encoders[column].fit_transform(df[column])

<ipython-input-31-434c06218d1f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = label_encoders[column].fit_transform(df[column])
<ipython-input-31-434c06218d1f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = label_encoders[column].fit_transform(df[column])
<ipython-input-31-434c06218d1f>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

In [40]:
df["Rating"].value_counts()

5    10369
0     1272
4     1241
3      368
1      210
2      174
Name: Rating, dtype: int64

In [32]:
X = df.drop(columns=['Rating'])
y = df['Rating']

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
model_xgb = xgb.XGBClassifier()
model_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [35]:
y_pred_xgb = model_xgb.predict(X_test)
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("XGBoost Accuracy:", accuracy_xgb)
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

XGBoost Accuracy: 0.7664099743307664
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.21      0.29       238
           1       0.67      0.10      0.17        41
           2       0.00      0.00      0.00        40
           3       0.20      0.04      0.06        78
           4       0.21      0.02      0.04       248
           5       0.79      0.97      0.87      2082

    accuracy                           0.77      2727
   macro avg       0.39      0.22      0.24      2727
weighted avg       0.68      0.77      0.70      2727



In [36]:
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001146 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2176
[LightGBM] [Info] Number of data points in the train set: 10907, number of used features: 14
[LightGBM] [Info] Start training from score -2.355970
[LightGBM] [Info] Start training from score -4.167261
[LightGBM] [Info] Start training from score -4.399320
[LightGBM] [Info] Start training from score -3.627279
[LightGBM] [Info] Start training from score -2.396429
[LightGBM] [Info] Start training from score -0.274717


LGBMClassifier()

In [37]:
y_pred_lgb = model_lgb.predict(X_test)
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)
print("LightGBM Accuracy:", accuracy_lgb)
print("LightGBM Classification Report:")
print(classification_report(y_test, y_pred_lgb))

LightGBM Accuracy: 0.7671433810047671
LightGBM Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.16      0.24       238
           1       0.40      0.10      0.16        41
           2       0.00      0.00      0.00        40
           3       0.20      0.03      0.05        78
           4       0.43      0.02      0.05       248
           5       0.78      0.98      0.87      2082

    accuracy                           0.77      2727
   macro avg       0.38      0.22      0.23      2727
weighted avg       0.69      0.77      0.69      2727



In [38]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

In [39]:
model_xgb_resampled = xgb.XGBClassifier()
model_xgb_resampled.fit(X_resampled, y_resampled)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [41]:
y_pred_xgb_final = model_xgb_resampled.predict(X_test)
accuracy_xgb_final = accuracy_score(y_test, y_pred_xgb_final)
print("Final XGBoost Accuracy:", accuracy_xgb_final)
print("Final XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb_final))

Final XGBoost Accuracy: 0.6674000733406674
Final XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.26      0.27       238
           1       0.14      0.15      0.14        41
           2       0.02      0.03      0.02        40
           3       0.07      0.06      0.07        78
           4       0.15      0.11      0.13       248
           5       0.79      0.83      0.81      2082

    accuracy                           0.67      2727
   macro avg       0.24      0.24      0.24      2727
weighted avg       0.65      0.67      0.66      2727



In [42]:
model_lgb_resampled = lgb.LGBMClassifier()
model_lgb_resampled.fit(X_resampled, y_resampled)

# Final Evaluation for LightGBM
y_pred_lgb_final = model_lgb_resampled.predict(X_test)
accuracy_lgb_final = accuracy_score(y_test, y_pred_lgb_final)
print("Final LightGBM Accuracy:", accuracy_lgb_final)
print("Final LightGBM Classification Report:")
print(classification_report(y_test, y_pred_lgb_final))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2479
[LightGBM] [Info] Number of data points in the train set: 49722, number of used features: 14
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
Final LightGBM Accuracy: 0.6574990832416575
Final LightGBM Classification Report:
              precision    recall  f1-score   support

           0       0.23      0.24      0.23       238
           1       0.15      0.22      0.17        41
           2       0.08      0.10      0.09        40
           3       0.07      0.05      0.06        78
           4       0.10      0.06 

In [43]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
import regex as re

In [44]:
model_xgb = xgb.XGBClassifier()
model_lgb = lgb.LGBMClassifier()

# Define resampling techniques
smote = SMOTE(random_state=42)
adasyn = ADASYN(random_state=42)
undersampler = RandomUnderSampler(random_state=42)

# Define parameter grids for hyperparameter tuning
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7]
}

param_grid_lgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7]
}

# Define pipelines for XGBoost and LightGBM models
pipeline_xgb = Pipeline([
    ('sampling', smote),
    ('classifier', GridSearchCV(model_xgb, param_grid_xgb, cv=3))
])

pipeline_lgb = Pipeline([
    ('sampling', adasyn),
    ('classifier', GridSearchCV(model_lgb, param_grid_lgb, cv=3))
])

# Train models
pipeline_xgb.fit(X_train, y_train)
pipeline_lgb.fit(X_train, y_train)


Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

Pipeline(steps=[('sampling', ADASYN(random_state=42)),
                ('classifier',
                 GridSearchCV(cv=3, estimator=LGBMClassifier(),
                              param_grid={'max_depth': [3, 5, 7],
                                          'n_estimators': [100, 200, 300]}))])

In [45]:
y_pred_xgb = pipeline_xgb.predict(X_test)
y_pred_lgb = pipeline_lgb.predict(X_test)

# Evaluation
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
accuracy_lgb = accuracy_score(y_test, y_pred_lgb)

print("XGBoost Accuracy:", accuracy_xgb)
print("XGBoost Classification Report:")
print(classification_report(y_test, y_pred_xgb))

print("LightGBM Accuracy:", accuracy_lgb)
print("LightGBM Classification Report:")
print(classification_report(y_test, y_pred_lgb))

# Print confusion matrix for XGBoost and LightGBM
print("Confusion Matrix (XGBoost):")
print(confusion_matrix(y_test, y_pred_xgb))

print("Confusion Matrix (LightGBM):")
print(confusion_matrix(y_test, y_pred_lgb))

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
XGBoost Accuracy: 0.707004033736707
XGBoost Classification Report:
              precision    recall  f1-score   support

           0       0.32      0.23      0.27       238
           1       0.29      0.17      0.22        41
           2       0.00      0.00      0.00        40
           3       0.08      0.05      0.06        78
           4       0.14      0.07      0.10       248
           5       0.79      0.89      0.84      2082

    accuracy                           0.71      2727
   macro avg       0.27      0.24      0.25      2727
weighted avg       0.65      0.71      0.68      2727

LightGBM Accuracy: 0.6894022735606894
LightGBM Classification Report:
              precision    recall  f1-score   support

           0       0.27      0.22      0.24       238
           1       0.16      0.15      0.15        41
           2       0.00    

In [46]:
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Define LightGBM parameters
params = {
    'num_leaves': 64,  # Increase num_leaves to address the warning
    'learning_rate': 0.1,
    'objective': 'multiclass',
    'metric': 'multi_error',
    'num_class': len(y_train.unique())  # Number of classes
}

# Initialize LightGBM model
model = lgb.LGBMClassifier(**params)

# Train the model
model.fit(X_resampled, y_resampled)

# Prediction
y_pred = model.predict(X_test)

# Evaluation
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004277 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2479
[LightGBM] [Info] Number of data points in the train set: 49722, number of used features: 14
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
[LightGBM] [Info] Start training from score -1.791759
Accuracy: 0.6897689768976898
Classification Report:
              precision    recall  f1-score   support

           0       0.29      0.26      0.27       238
           1       0.15      0.15      0.15        41
           2       0.03      0.03      0.03        40
           3       0.05      0.04      0.04        78


In [58]:
import time
start_time = time.time()

from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=300, max_depth=None,min_samples_split=5,max_features='sqrt',random_state=0)
rfc.fit(X_train, y_train)
# calculate accuracy of class predictions
y_pred_class = rfc.predict(X_test)
print(classification_report(y_test,y_pred_class))
print(confusion_matrix(y_test,y_pred_class))


print("--- %s seconds ---" % (time.time() - start_time))

              precision    recall  f1-score   support

           0       0.57      0.14      0.22       238
           1       0.20      0.05      0.08        41
           2       0.00      0.00      0.00        40
           3       0.00      0.00      0.00        78
           4       0.50      0.00      0.01       248
           5       0.78      0.99      0.87      2082

    accuracy                           0.77      2727
   macro avg       0.34      0.20      0.20      2727
weighted avg       0.69      0.77      0.69      2727

[[  33    5    0    2    0  198]
 [   3    2    1    0    0   35]
 [   1    0    0    0    1   38]
 [   1    0    0    0    0   77]
 [   0    2    0    0    1  245]
 [  20    1    1    0    0 2060]]
--- 8.986149787902832 seconds ---


In [48]:
import time
start_time = time.time()


from sklearn.ensemble import GradientBoostingClassifier
a = GradientBoostingClassifier(max_depth=200, learning_rate=0.2, n_estimators=50, random_state=52)
a.fit(X_train, y_train)
# calculate accuracy of class predictions
y_pred_class = a.predict(X_test)
print(classification_report(y_test,y_pred_class))
print(confusion_matrix(y_test,y_pred_class))

print("--- %s seconds ---" % (time.time() - start_time))

              precision    recall  f1-score   support

           0       0.34      0.27      0.30       238
           1       0.15      0.12      0.13        41
           2       0.05      0.03      0.03        40
           3       0.11      0.05      0.07        78
           4       0.15      0.10      0.12       248
           5       0.80      0.88      0.84      2082

    accuracy                           0.71      2727
   macro avg       0.27      0.24      0.25      2727
weighted avg       0.66      0.71      0.68      2727

[[  65    8    0    1   17  147]
 [   3    5    5    3    1   24]
 [   3    1    1    1    2   32]
 [   4    2    2    4   10   56]
 [  15    6    0    9   24  194]
 [ 101   12   11   19  109 1830]]
--- 78.67556309700012 seconds ---


In [49]:
import time
start_time = time.time()

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
print("ADA")
xgb = AdaBoostClassifier(n_estimators=200, random_state=5,learning_rate=0.5)
xgb_pred = xgb.fit(X_train, y_train).predict(X_test)
print(classification_report(y_test,xgb_pred))
print(confusion_matrix(y_test,xgb_pred))

print("--- %s seconds ---" % (time.time() - start_time))

ADA
              precision    recall  f1-score   support

           0       0.42      0.04      0.08       238
           1       0.36      0.12      0.18        41
           2       0.00      0.00      0.00        40
           3       0.00      0.00      0.00        78
           4       0.00      0.00      0.00       248
           5       0.77      0.99      0.87      2082

    accuracy                           0.76      2727
   macro avg       0.26      0.19      0.19      2727
weighted avg       0.63      0.76      0.67      2727

[[  10    4    0    2    0  222]
 [   0    5    1    0    0   35]
 [   1    1    0    0    0   38]
 [   4    1    2    0    0   71]
 [   0    0    1    1    0  246]
 [   9    3    2    1    0 2067]]
--- 7.738793134689331 seconds ---


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [50]:
import time
start_time = time.time()

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
# 2. instantiate a logistic regression model
logreg = LogisticRegression( solver='saga',multi_class='ovr',C=500.0)
# 3. train the model using X_train_dtm
x=logreg.fit(X_train, y_train)
# 4. make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test)
print(classification_report(y_test,y_pred_class))
print(confusion_matrix(y_test,y_pred_class))

print("--- %s seconds ---" % (time.time() - start_time))

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       238
           1       0.00      0.00      0.00        41
           2       0.00      0.00      0.00        40
           3       0.00      0.00      0.00        78
           4       0.00      0.00      0.00       248
           5       0.76      1.00      0.87      2082

    accuracy                           0.76      2727
   macro avg       0.13      0.17      0.14      2727
weighted avg       0.58      0.76      0.66      2727

[[   0    0    0    0    0  238]
 [   0    0    0    0    0   41]
 [   0    0    0    0    0   40]
 [   0    0    0    0    0   78]
 [   0    0    0    0    0  248]
 [   0    0    0    0    0 2082]]
--- 2.9310219287872314 seconds ---


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. 

In [59]:
import time
start_time = time.time()

import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
r1 = AdaBoostClassifier(n_estimators=200, random_state=5,learning_rate=0.5)
r2 = RandomForestClassifier(n_estimators=300, max_depth=None,min_samples_split=5,max_features='sqrt',random_state=0)


er = VotingClassifier([('lr', r1),('rf', r2)],voting="soft")

x=er.fit(X_train, y_train)

y_pred_class = er.predict(X_test)
print(classification_report(y_test,y_pred_class))
print(confusion_matrix(y_test,y_pred_class))

print("--- %s seconds ---" % (time.time() - start_time))

              precision    recall  f1-score   support

           0       0.57      0.14      0.22       238
           1       0.20      0.05      0.08        41
           2       0.00      0.00      0.00        40
           3       0.00      0.00      0.00        78
           4       0.50      0.00      0.01       248
           5       0.78      0.99      0.87      2082

    accuracy                           0.77      2727
   macro avg       0.34      0.20      0.20      2727
weighted avg       0.69      0.77      0.69      2727

[[  33    5    0    2    0  198]
 [   3    2    1    0    0   35]
 [   1    0    0    0    1   38]
 [   1    0    0    0    0   77]
 [   0    2    0    0    1  245]
 [  20    1    1    0    0 2060]]
--- 12.998268365859985 seconds ---
